In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'faridabadonlinejournal_in'
c = Crawl()  # creating object
cl_data = dashboard['core_web_india']
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_fetch_rss_page = 0
publish_source = 'faridabadonlinejournal.in'
country = 'India'
language = 'English'
images_path = []

rss_page = c.download_page('http://www.faridabadonlinejournal.in/feed')

for i in rss_page.split('<item>')[1:]:
    
    source_link = c.scrap('<link>(.*?)<', i)
    
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue       
        
    time.sleep(random.randint(1,3))        
        
    page = c.download_page(source_link)
        
    if page.startswith('Unable to fetch'):
        logger.info(page) # writes error message with error code
        unable_to_fetch_url += 1
        continue  
    
    source_headline = c.scrap('<title>(.*?)<', page)
    source_headline = re.sub('&ldquo;|&rdquo;|&rsquo;|&mdash;|&lsquo;|&#8211;|&#8217;|&#8220;|&#160;|&amp;|&bull;|&#124;|&#8221;|&#8216;|&middot;|&#038;|&trade;|&trade;|&ndash;|&minus;|&#8243;','',source_headline,flags=re.S)
    
      #skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

    logger.info(f'Fetching {source_link}\n')
    
     # Date and time
    pub_date, publish_time = '', ''

    try:
        date_time_str = c.scrap('<pubDate>(.*?)</', i)
        date_time_str = date_time_str.replace('+0000','')
        date_time_str = re.sub('^\w.*?,|:|\s','',date_time_str) 
        date_time_obj = datetime.strptime(date_time_str, '%d%b%Y%H%M%S')
        ist_date_time = date_time_obj + timedelta(hours = 0, minutes = 0)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
    except:
        pass
   
        # skip null date
        
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')
        skipped_due_to_date += 1
        continue 
        
    if pub_date != created_on:
        break
        
    journalist='' #journalist is not  present in this website
    if not journalist: journalist = 'NA'
    
    list_regex = ['<span\s*class="head-year">(.*?)<p\s*class="caps">',
                  '<div\s*class="quotes">.*?<div\s*>(.*?)<p\s*class="caps">',
                  '<span\s*class="head-year">(.*?)<aside\s*id="sidebar">'
                ]
    for x in list_regex:
        source_content = c.scrap(x,page)
        source_content = re.sub('&ldquo;|&rdquo;|&rsquo;|&mdash;|&lsquo;|&#8211;|&#8217;|&#8220;|&#160;|&amp;|&bull;|&#124;|&#8221;|&#8216;|&middot;|&#038;|&trade;|&ndash;|&minus;|&#8243;','',source_content,flags=re.S)
        source_content = c.strip_html(source_content)
        print(source_content)
        if source_content:                              
            break
        
     #current date and time 
    harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
    temp_link = source_link

        # headline and content 
    headline = source_headline
    content = source_content

        # overall_tonality
    overall_tonality = ''

        # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

        # image_urls
    image_urls = []

    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content
                } 

    cl_data.insert_one(clientdata)  
    no_of_data += 1
    
logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch rss url: {unable_to_fetch_rss_page}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')
           
    

2021

					The Global Polysorbate Market size was valued at USD 1,015.0 Million in 2019 and is anticipated to reach USD 1,425.3 Million by 2027 at a CAGR of 4.7%.
The growing demand for cosmetic products and increasing application of polysorbate in the food industry are driving the demand for the market.
The study has been conducted on the basis of inputs from the industry leaders. Thus, a conscious effort to dive deep and unearth hard to find information pertaining to the growing market landscape and growth prospects over the next few years is clearly visible in the study. The market intelligence report further comprises of discussions about the major vendors operating in the Polysorbate space
Get Free Sample PDF Copy of this Report At: https://www.emergenresearch.com/request-sample/201
The section on the competitive landscape offers valuable and actionable insights related to the business sphere of the Polysorbate market, covering extensive profiling of the key market players. The re

2021

					A closer look at the aspects including but not limited to Adipose-derived Stem Cell market segmentation by the end-user, end-use, geography, type, and application forms an integral part of the research report.
Adipose tissue-derived stem cells are currently being used for a broad spectrum of applications such as regenerative medicine, tissue engineering, cell therapy, and stem cell differentiation studies. ADSCs have a massive potential for regenerative medicine and demonstrate more robust yields than other stem cells. This can be attributed to its anti-inflammatory, immunomodulatory, anti-scarring, and anti-apoptotic properties, among others.
The study has been conducted on the basis of inputs from the industry leaders. Thus, a conscious effort to dive deep and unearth hard to find information pertaining to the growing market landscape and growth prospects over the next few years is clearly visible in the study. The market intelligence report further comprises of discussion

2021

					The Global Drone Delivery Service Market was valued at USD 520.0 Million in 2019 and is forecasted to reach USD 9.51 Billion by 2027 at a CAGR of 44.2%. The need to deliver goods faster has boosted the growth in demand for drone logistics. This, in turn has affected the market positively on a global scale.
The detailed market intelligence report on the Drone Delivery Service market applies the best of both primary and secondary research to weighs upon the competitive landscape and the prominent market players expected to dominate the Drone Delivery Service market for the forecast period, 2020  2027. The study not only scans through the company profile of the major vendors but also analyses their winning strategies to give business owners, stakeholders and field marketing personnel a competitive edge over others operating in the same space.
The growing usage of drones across commercial applications like food delivery and medical aids delivery is one of the key factors project

2021

					The study methodologies used to examine the Directed Energy Weapons market for the forecast period, 2020  2027 further classifies the industry by type, geography, end-use and end-user to add more precision and bring to light factors responsible for augmenting business development.
The Global Directed Energy Weapons (DEW) Market is forecasted to be worth USD 93.20 billion by 2027, according to a current analysis by Emergen Research. The market is remarkably being driven by the factors such as increasing adoption of the directed energy weapons to destroy or damage the rockets, drones, mortar-launched explosives and any other aerial threats, higher investment in the military bodies  defense intelligence sectors, and technological development in the laser-based target focused weapons.
The report on the Directed Energy Weapons market gives an in-depth statistical analysis to examine the fastest growing sectors in the market while speculating the demand and supply, consumption pow

2021

					The study methodologies used to examine the Wind Energy market for the forecast period, 2020  2027 further classifies the industry by type, geography, end-use and end-user to add more precision and bring to light factors responsible for augmenting business development.
The globalWind Energy Marketwill be worth USD 167.15 Billion by 2027, according to a current analysis by Emergen Research. The growth of this market can be attributed to the rapid urbanization coupled with the rising demand for electricity in the developing economies. Rising initiatives of the government to increase the adoption of sustainable sources of power generation such as solar energy and wind energy in order to reduce the emission levels and enhance the quality of air is anticipated to drive the growth of the segment.]
The report on the Wind Energy market gives an in-depth statistical analysis to examine the fastest growing sectors in the market while speculating the demand and supply, consumption powe

...

2021

					Huizhou Calvindude Technology Co., Ltd introduces state-of-the-art automated assembly line machines for use in various electronics and manufacturing industries.
Huizhou Calvindude Technology Co., Ltd is one of the most prestigious companies known for its optimum services and quality product. The company is known for its best automatic assembly machines, which feature advanced technology and consume less energy. They keep in line with the modern innovations and developments and always incorporates innovative technological methods in their manufacturing process. It has always expanded its fleet of machines and invested in the latest technology. These efforts have resulted in machines of the highest quality that meet national and international standards. All of their devices are of superior quality, proper specifications, and are manufactured according to customer instructions. These machines are in mammoth demand in the market not only in China but also all across the globe be

2021

					Market Size – USD 2.10 Billion in 2019, Market Growth  CAGR of 44.1%, Market Trends – The rise in the adoption of advanced operational technology in several industries.

The increasing IoT penetration and growing government initiatives to improve the retail industry are driving the demand for the market.
The global Beacon Technology Market is forecasted to be worth USD 35.15 Billion by 2027, according to a current analysis by Emergen Research. The beacon technology market is anticipated to grow substantially owing to the growing Internet of Things technology penetration, increasing investments in proximity marketing, and rising demand for sensor-based devices in different industries. The growth in organized retail stores provides the players functioning in the beacon technology market with sufficient growth prospects. With the support of government organized retail outlets are growing substantially, such as supermarkets, department stores, and hypermarkets, which is expected

2021

					Market Size – USD 795.0 Million in 2019, Market Growth  CAGR of 34.3%, Market Trends – The rise in the application of this technology to reduce traffic congestions.
The growing adoption of IoT-enabled lighting solutions and the rising use of smart LED lighting systems for wireless connectivity is driving the demand for the market.
The globalHuman Centric Lighting Marketis forecasted to be worth USD 7,641.9 Million by 2027, according to a current analysis by Emergen Research.The growing smart LED lighting adoption in the retail, commercial, and residential sectors is forecasted to propel the human-centric lighting market during the forecast period. Several companies strategy to make more of their goods human-centered, adapting to the specific needs of human sociology and psychology, is another key driver for the progress of the human-centric lighting industry. As per Accenture plc, companies need to move from a product-focused strategy towards a human-centric approach to buil

2021

					Market Size – USD 19.93 Billion in 2019, Market Growth  CAGR of 19.6%, Market trends – The infiltration of the internet in the Healthcare sector.
The growing initiative to promote the adoption of healthcare information and the rising advent of cloud will drive the demand for Big Data in the Healthcare market.
The globalBig Data in Healthcare Marketis forecasted to be worth USD 78.03 Billion by 2027, according to a current analysis by Emergen Research. An increasing cost of healthcare, a rise in chronic diseases, a growing geriatric population, and a fall in reimbursement cost will propel the demand for the market.
Request For The Sample Copy: https://www.emergenresearch.com/request-sample/236
Healthcare sectors are under immense pressure to focus on investment improving resource and outcomes management. Deployment of analytical tools, artificial intelligence, and machine learning techniques on the growing amount of data to enhance revenue, reduce healthcare costs, and person

2021

					New name for 45-year-old health services provider reflects expansion of care services and growing commitment to Bay Area and Northern California communities.
Larkspur, CA, USA  April 1, 2021  Hospice by the Bay, the trusted provider of hospice, palliative care, and in-home health care services, today announced that it has taken a new name, By the Bay Health. The name change is designed to more accurately reflect the organizations current breadth of services, which include hospice care, palliative care, comfort care, grief counseling, pediatric care, and skilled home health care services.

Founded 45 years ago as Hospice of Marin, By the Bay Health is the oldest non-profit hospice in California and the second oldest in the United States. The organization has since been meeting the rising demand for more options with regards to home health care services beyond its hospice care.
Our commitment to provide end-of-life care will always be at the core of this organization, says Kit